**Задание для лабораторной №7:**  
1. Построить базу знаний в любой предметной области, сформировав массив документов (например, публикаций или файлов на каком-либо портале или набор файлов на компьютере или сервере и т.п.) и построить граф знаний об этих документах
2. Реализовать сервис краткого описания каждого документа в базе знаний (абстракт, автореферат - summary)
3. Реализовать сервис поиска по запросу внутри базы знаний (вопрос-ответный модуль)

### 0. Загрузка данных

Данные взяты с сайта: https://ru.wikipedia.org/wiki/Категория:Знаменитые_животные_по_алфавиту

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
import time

In [ ]:
os.makedirs('famous_animals', exist_ok=True)

In [ ]:
base_url = 'https://ru.wikipedia.org'
category_url = f'{base_url}/wiki/Категория:Знаменитые_животные_по_алфавиту'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

In [ ]:
response = requests.get(category_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
content_div = soup.find('div', {'id': 'mw-pages'})
if content_div:
    animal_links = content_div.find_all('a')
else:
    animal_links = []

In [ ]:
print(f"Найдено животных: {len(animal_links)}")

for link in animal_links:
    animal_name = link.text
    animal_href = link.get('href')
    if not animal_href or not animal_href.startswith('/wiki/'):
        continue
    animal_url = base_url + animal_href

    try:

        time.sleep(1)

        animal_response = requests.get(animal_url, headers=headers)
        if animal_response.status_code == 429:
            print(f"Too many requests for {animal_name}. Waiting...")
            time.sleep(10)
            continue

        animal_soup = BeautifulSoup(animal_response.content, 'html.parser')

        paragraphs = animal_soup.find_all('p')
        description = ''
        for p in paragraphs:
            if p.text.strip() and len(p.text.strip()) > 50:
                description = p.text.strip()
                break

        infobox = animal_soup.find('table', {'class': 'infobox'})
        animal_type = "Неизвестно"
        birth_date = "Неизвестно"

        if infobox:
            rows = infobox.find_all('tr')
            for r in rows:
                if r.th:
                    header = r.th.text.strip()
                    if "вид" in header.lower():
                        animal_type = r.td.text.strip()
                    if "дата рождения" in header.lower() or "дата появления" in header.lower():
                        birth_date = r.td.text.strip()

        # Если описание не найдено, пишем заглушку
        if not description:
            description = "Описание недоступно."

        animal_data = {
            'Название': animal_name,
            'Описание': description,
            'Вид животного': animal_type,
            'Дата рождения': birth_date,
            'Ссылка': animal_url
        }

        filename = f"famous_animals/{animal_name.replace('/', '_')}.json"
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(animal_data, f, ensure_ascii=False, indent=4)

        print(f"Сохранено: {animal_name}")

    except Exception as e:
        print(f"Ошибка при обработке {animal_name}: {e}")

Найдено животных: 207
Сохранено: Список ниже может не отражать последних изменений.
Сохранено: Искать по категории
Сохранено: Случайная страница в категории
Сохранено: Абсент (конь)
Сохранено: Абу-ль-Аббас (слон)
Сохранено: Абутью
Сохранено: Адвайта (черепаха)
Сохранено: Алекс (попугай)
Сохранено: Аллентонский бегемот
Сохранено: Амадей (тигр)
Сохранено: Амур и Тимур
Сохранено: Ангалифу
Сохранено: Анилин (конь)
Сохранено: Анна Савельевна
Сохранено: Ань Ань
Сохранено: Ань-Ань
Сохранено: Арго (волк)
Сохранено: Байерли Тюрк
Сохранено: Балто
Сохранено: Барни (собака)
Сохранено: Барри (собака)
Сохранено: Барт (медведь)
Сохранено: Баська Мурманская
Сохранено: Батыр (слон)
Сохранено: Бахати (львенок)
Сохранено: Белка и Стрелка
Сохранено: Берёзовский мамонт
Сохранено: Берт (крокодил)
Сохранено: Блонди
Сохранено: Бо Обама
Сохранено: Боби (пёс)
Сохранено: Бой (пудель)
Сохранено: Бокито (горилла)
Сохранено: Большой Шлёпа
Сохранено: Борец (медведь)
Сохранено: Борис (собака)
Сохранено: Бричес (обезь

In [ ]:
folder_path = 'famous_animals'

# Список служебных слов
bad_words = ['список', 'искать', 'страница', 'категория', 'изменений', 'случайная']

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            name = data.get('Название', '').lower()

            if any(bad_word in name for bad_word in bad_words):
                print(f"Удаляю: {filename}")
                os.remove(file_path)

    except Exception as e:
        print(f"Ошибка при обработке {filename}: {e}")

Удаляю: Искать по категории.json
Удаляю: Случайная страница в категории.json
Удаляю: Список ниже может не отражать последних изменений..json


In [ ]:
folder_path = 'famous_animals'

missing_description = 0
missing_type = 0
missing_birth = 0
total_files = 0

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                total_files += 1

                if data.get('Описание', '') == "Описание недоступно.":
                    missing_description += 1
                if data.get('Вид животного', '') == "Неизвестно":
                    missing_type += 1
                if data.get('Дата рождения', '') == "Неизвестно":
                    missing_birth += 1

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

print(f"Всего файлов: {total_files}")
print(f"Без описания: {missing_description}")
print(f"Без указания вида: {missing_type}")
print(f"Без даты рождения: {missing_birth}")

Всего файлов: 293
Без описания: 0
Без указания вида: 67
Без даты рождения: 61


In [ ]:
import re

folder_path = 'famous_animals'

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            name = data.get('Название', '')
            animal_type = data.get('Вид животного', 'Неизвестно')

            match = re.match(r'(.+?)\s*\((.+?)\)', name)
            if match:
                clean_name = match.group(1).strip()
                type_in_brackets = match.group(2).strip()

                data['Название'] = clean_name

                if animal_type == "Неизвестно" or not animal_type:
                    data['Вид животного'] = type_in_brackets

                new_filename = f"{clean_name.replace('/', '_')}.json"
                new_path = os.path.join(folder_path, new_filename)

                os.remove(file_path)
                with open(new_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)

            else:
                with open(file_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

In [ ]:
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            animal_type = data.get('Вид животного', '')
            animal_year = data.get('Дата рождения', '')

            # Если вид животного "Неизвестно" — удаляем файл
            if animal_type.strip().lower() == 'неизвестно':
                print(f"Удаляю файл без вида: {filename}")
                os.remove(file_path)

            if animal_year.strip().lower() == 'неизвестно':
                print(f"Удаляю файл без года рождения: {filename}")
                os.remove(file_path)


        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

Удаляю файл без вида: Берёзовский мамонт.json
Удаляю файл без года рождения: Берёзовский мамонт.json
Ошибка при обработке Берёзовский мамонт.json: [Errno 2] No such file or directory: 'famous_animals/Берёзовский мамонт.json'
Удаляю файл без вида: Аллентонский бегемот.json
Удаляю файл без года рождения: Аллентонский бегемот.json
Ошибка при обработке Аллентонский бегемот.json: [Errno 2] No such file or directory: 'famous_animals/Аллентонский бегемот.json'
Удаляю файл без вида: Баська Мурманская.json
Удаляю файл без года рождения: Баська Мурманская.json
Ошибка при обработке Баська Мурманская.json: [Errno 2] No such file or directory: 'famous_animals/Баська Мурманская.json'
Удаляю файл без вида: Канзи.json
Удаляю файл без вида: Гуруваюр Кешаван.json
Удаляю файл без вида: Обайш.json
Удаляю файл без года рождения: Обайш.json
Ошибка при обработке Обайш.json: [Errno 2] No such file or directory: 'famous_animals/Обайш.json'
Удаляю файл без года рождения: Ларри.json
Удаляю файл без года рождения

In [ ]:
print(f"Всего файлов: {total_files}")
print(f"Без описания: {missing_description}")
print(f"Без указания вида: {missing_type}")
print(f"Без даты рождения: {missing_birth}")

Всего файлов: 293
Без описания: 0
Без указания вида: 67
Без даты рождения: 61


In [ ]:
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            if 'Страна' not in data:
                data['Страна'] = "Неизвестно"

                with open(file_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)

                print(f"Добавлена страна в: {filename}")

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

Добавлена страна в: Кузя.json
Добавлена страна в: Джонатан.json
Добавлена страна в: Железный Ковбой.json
Добавлена страна в: Луга.json
Добавлена страна в: Абу-ль-Аббас.json
Добавлена страна в: Диего.json
Добавлена страна в: Арго.json
Добавлена страна в: Большой Шлёпа.json
Добавлена страна в: Мэйджор.json
Добавлена страна в: Пегги.json
Добавлена страна в: Нильс Улаф.json
Добавлена страна в: Оскар.json
Добавлена страна в: Анилин.json
Добавлена страна в: Годзилла.json
Добавлена страна в: Дик.json
Добавлена страна в: Линь Ван.json
Добавлена страна в: Батыр.json
Добавлена страна в: Манч.json
Добавлена страна в: Балто.json
Добавлена страна в: Земира.json
Добавлена страна в: Майкл.json
Добавлена страна в: Виннипег.json
Добавлена страна в: Зарафа.json
Добавлена страна в: Борец.json
Добавлена страна в: Бокито.json
Добавлена страна в: Гли.json
Добавлена страна в: Барри.json
Добавлена страна в: Гувер.json
Добавлена страна в: Дина.json
Добавлена страна в: Барни.json
Добавлена страна в: Мотти.json


In [ ]:
import requests

folder_path = 'famous_animals'
base_url = 'https://ru.wikipedia.org'
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            if data.get('Страна', '') != "Неизвестно":
                continue

            url = data['Ссылка']
            time.sleep(1)
            response = requests.get(url, headers=headers)

            if response.status_code != 200:
                print(f"Ошибка запроса для {url}")
                continue

            soup = BeautifulSoup(response.content, 'html.parser')

            infobox = soup.find('table', {'class': 'infobox'})
            country = "Неизвестно"

            if infobox:
                rows = infobox.find_all('tr')
                for row in rows:
                    if row.th and 'страна' in row.th.text.lower():
                        country = row.td.text.strip()
                        break

            data['Страна'] = country

            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

            print(f"Обновлена страна для: {data['Название']} — {country}")

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

Обновлена страна для: Кузя — Россия
Обновлена страна для: Джонатан — Остров Св. Елены
Обновлена страна для: Железный Ковбой — Неизвестно
Обновлена страна для: Луга — Россия
Обновлена страна для: Абу-ль-Аббас — Неизвестно
Обновлена страна для: Диего — Неизвестно
Обновлена страна для: Арго — СССР
Обновлена страна для: Большой Шлёпа — Россия
Обновлена страна для: Мэйджор — Неизвестно
Обновлена страна для: Пегги — Великобритания
Обновлена страна для: Нильс Улаф — Неизвестно
Обновлена страна для: Оскар — США
Обновлена страна для: Анилин — СССР
Обновлена страна для: Годзилла — 
Обновлена страна для: Дик — СССР
Обновлена страна для: Линь Ван — Неизвестно
Обновлена страна для: Батыр — СССР Казахстан
Обновлена страна для: Манч — Венгрия
Обновлена страна для: Балто — США
Обновлена страна для: Земира — Российская империя
Обновлена страна для: Майкл — США
Обновлена страна для: Виннипег — Неизвестно
Обновлена страна для: Зарафа — Неизвестно
Обновлена страна для: Борец — Российская империя (до 1917 

In [ ]:
country_mapping = {
    'СССР': 'Россия',
    'Российская империя': 'Россия',
    'Англия': 'Великобритания',
    'Шотландия': 'Великобритания',
    'Уэльс': 'Великобритания',
    'Северная Ирландия': 'Великобритания',
    'Ирландия': 'Великобритания',
    'Остров Св. Елены': 'Великобритания',
    'Нацистская Германия': 'Германия',
    'Великобритания →  США': 'Великобритания',
    'Древний Египет': 'Египет',
    'Италия,  Швейцария': 'Швейцария',
    'Китай  Япония': 'Китай',
    'Соединённые Штаты Америки': 'США',
}

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            country = data.get('Страна', '').strip()

            for old, new in country_mapping.items():
                if old.lower() in country.lower():
                    print(f"Изменяем для {data['Название']}: {country} ➔ {new}")
                    data['Страна'] = new
                    break

            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

Изменяем для Джонатан: Остров Св. Елены ➔ Великобритания
Изменяем для Арго: СССР ➔ Россия
Изменяем для Анилин: СССР ➔ Россия
Изменяем для Дик: СССР ➔ Россия
Изменяем для Батыр: СССР Казахстан ➔ Россия
Изменяем для Земира: Российская империя ➔ Россия
Изменяем для Борец: Российская империя (до 1917 г.), СССР ➔ Россия
Изменяем для Дина: СССР ➔ Россия
Изменяем для Неджем: Древний Египет ➔ Египет
Изменяем для Кашмир: Ирландия ➔ Великобритания
Изменяем для Грейфрайерс Бобби: Шотландия ➔ Великобритания
Изменяем для Олд Джок: Англия ➔ Великобритания
Изменяем для Абсент: СССР ➔ Россия
Изменяем для Блонди: Нацистская Германия ➔ Германия
Изменяем для Марьям: СССР ➔ Россия
Изменяем для Ань-Ань: СССР ➔ Россия
Изменяем для Маша: СССР →  Россия (город  Ярославль) ➔ Россия
Изменяем для Джо Мартин: Соединённые Штаты Америки ➔ США
Изменяем для Кинули: СССР ➔ Россия
Изменяем для Лайка: СССР ➔ Россия


In [ ]:
corrections = {
    "Нильс Улаф": "Норвегия",
    "Линь Ван": "Тайвань",
    "Виннипег": "Канада",
    "Зарафа": "Франция",
    "Бокито": "Нидерланды",
    "Гли": "Турция",
    "Барни": "США",
    "Клара": "Германия",
    "Ангалифу": "ЮАР",
    "Лолита": "США",
    "Непотопляемый Сэм": "Германия",
    "Мисс Бизли": "США",
    "Адвайта": "Индия",
    "Бой": "Великобритания",
    "Крымский Том": "Россия",
    "Долли": "Великобритания",
    "Джуди": "Великобритания",
    "Кадо": "США",
    "Камиль": "Франция",
    "Конни Полгрейв": "Россия",
    "Магава": "Камбоджа",
    "Алекс": "США",
    "Мусташ": "Франция",
    "Муизза": "Египет",
    "Дьюи Читатель Книг": "США",
    "Джамбо": "Великобритания",
    "Кейко": "Исландия",
    "Буджан Сэнан": "Турция",
    "Пауль": "Германия",
    "Бо Обама": "США",
    "Войтек": "Польша",
    "Оззи": "США",
    "Железный Ковбой": "Россия",
    "Абу-ль-Аббас": "Иран",
    "Диего": "Эквадор",
    "Мэйджор": "США",
    "Годзилла": "Украина"
}

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            name = data.get('Название', '')

            if name in corrections:
                old_country = data.get('Страна', '')
                new_country = corrections[name]

                # Обновляем страну
                data['Страна'] = new_country

                with open(file_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, ensure_ascii=False, indent=4)

                print(f"Обновлено для {name}: {old_country} ➔ {new_country}")

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

Обновлено для Железный Ковбой: Неизвестно ➔ Россия
Обновлено для Абу-ль-Аббас: Неизвестно ➔ Иран
Обновлено для Диего: Неизвестно ➔ Эквадор
Обновлено для Мэйджор: Неизвестно ➔ США
Обновлено для Нильс Улаф: Неизвестно ➔ Норвегия
Обновлено для Годзилла:  ➔ Украина
Обновлено для Линь Ван: Неизвестно ➔ Тайвань
Обновлено для Виннипег: Неизвестно ➔ Канада
Обновлено для Зарафа: Неизвестно ➔ Франция
Обновлено для Бокито: Неизвестно ➔ Нидерланды
Обновлено для Гли: Неизвестно ➔ Турция
Обновлено для Барни: Неизвестно ➔ США
Обновлено для Клара: Неизвестно ➔ Германия
Обновлено для Ангалифу: Неизвестно ➔ ЮАР
Обновлено для Лолита: Неизвестно ➔ США
Обновлено для Непотопляемый Сэм: Неизвестно ➔ Германия
Обновлено для Мисс Бизли: Неизвестно ➔ США
Обновлено для Адвайта: Неизвестно ➔ Индия
Обновлено для Бой: Неизвестно ➔ Великобритания
Обновлено для Крымский Том: Неизвестно ➔ Россия
Обновлено для Долли: Неизвестно ➔ Великобритания
Обновлено для Джуди: Неизвестно ➔ Великобритания
Обновлено для Кадо: Неизвес

In [ ]:
bad_countries = ['Великобритания → США', 'Италия', 'Швейцария']

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            country = data.get('Страна', '').strip()

            if country in bad_countries:
                print(f"Удаляю файл {filename} (страна: {country})")
                os.remove(file_path)

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

In [ ]:
countries = set()

missing_description = []
missing_type = []
missing_birth = []
missing_country = []

total_files = 0

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            total_files += 1

            country = data.get('Страна', '').strip()
            if country:
                countries.add(country)
            if country.lower() == 'неизвестно' or not country:
                missing_country.append(filename)

            # Проверка на пропуски
            if data.get('Описание', '').strip() == "Описание недоступно." or not data.get('Описание', '').strip():
                missing_description.append(filename)

            if data.get('Вид животного', '').strip().lower() == 'неизвестно' or not data.get('Вид животного', '').strip():
                missing_type.append(filename)

            if data.get('Дата рождения', '').strip().lower() == 'неизвестно' or not data.get('Дата рождения', '').strip():
                missing_birth.append(filename)

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

print(f"Всего файлов: {total_files}\n")

print("Уникальные страны:")
for c in sorted(countries):
    print("-", c)

print("\nФайлы с пропущенным описанием:", len(missing_description))

print("Файлы с пропущенным видом животного:", len(missing_type))

print("Файлы с пропущенной датой рождения:", len(missing_birth))

print("Файлы с пропущенной страной:", len(missing_country))

Всего файлов: 126

Уникальные страны:
- Австралия
- Бельгия
- Бразилия
- Бурунди
- Великобритания
- Великобритания →  США
- Венгрия
- Германия
- Египет
- Израиль
- Индия
- Иран
- Исландия
- Италия,  Швейцария
- Камбоджа
- Канада
- Китай  Япония
- Нидерланды
- Норвегия
- ОАЭ
- Польша
- Португалия
- Россия
- США
- Таиланд
- Тайвань
- Турция
- Украина
- Франция
- Чехия
- Эквадор
- ЮАР
- Япония

Файлы с пропущенным описанием: 0
Файлы с пропущенным видом животного: 0
Файлы с пропущенной датой рождения: 0
Файлы с пропущенной страной: 0


In [ ]:
import re
import os
import json

def extract_year(text):
    """
    Извлекает первый нормальный год (4 цифры) из текста с учетом различных вариантов форматов дат.
    Если года нет, возвращает None.
    """
    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(r'[^0-9а-яА-ЯёЁ\s]', '', text)

    # Регулярное выражение для нахождения года
    years = re.findall(r'\b\d{4}\b', text)

    if years:
        return years[0]

    if 'по' in text:
        range_years = re.findall(r'\b\d{4}\b', text)
        if range_years:
            return range_years[0]

    if 'примерно' in text or '?' in text:
        possible_years = re.findall(r'\b\d{4}\b', text)
        if possible_years:
            return possible_years[0]

    if 'не позднее' in text:
        possible_years = re.findall(r'\b\d{4}\b', text)
        if possible_years:
            return possible_years[0]

    return None

folder_path = 'famous_animals'

unique_years = set()

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        path = os.path.join(folder_path, filename)

        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        birth_date = data.get('Дата рождения', '').strip()

        year = extract_year(birth_date)

        if year:
            unique_years.add(year)

unique_years = sorted(unique_years)
print(f"Найдено уникальных годов: {len(unique_years)}")
for year in unique_years:
    print("-", year)

Найдено уникальных годов: 77
- 1479
- 1499
- 1638
- 1738
- 1750
- 1778
- 1799
- 1800
- 1830
- 1832
- 1847
- 1849
- 1855
- 1859
- 1860
- 1863
- 1868
- 1886
- 1914
- 1915
- 1917
- 1919
- 1924
- 1933
- 1935
- 1936
- 1939
- 1941
- 1942
- 1944
- 1945
- 1951
- 1952
- 1954
- 1955
- 1957
- 1961
- 1963
- 1970
- 1971
- 1972
- 1973
- 1974
- 1976
- 1977
- 1978
- 1981
- 1983
- 1985
- 1987
- 1988
- 1989
- 1990
- 1991
- 1992
- 1994
- 1996
- 1998
- 1999
- 2000
- 2001
- 2002
- 2004
- 2005
- 2006
- 2007
- 2008
- 2009
- 2010
- 2011
- 2012
- 2013
- 2015
- 2017
- 2018
- 2019
- 2024


In [ ]:
import os
import json

folder_path = 'famous_animals'

years = set()

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        year = data.get('Дата рождения', '').strip()

        if year and year != "Неизвестно":
            years.add(year)

years = sorted(years)
print(f"Найдено уникальных годов: {len(years)}")
for year in years:
    print("-", year)

Найдено уникальных годов: 123
- 1 мая 2001(2001-05-01)
- 1 октября 2009(2009-10-01) (15 лет)
- 10 декабря 2005
- 10 июля 2024 (5 месяцев)
- 10 октября 2005(2005-10-10)
- 10 сентября 2004(2004-09-10)
- 10 февраля 1919(1919-02-10)
- 11 июля 1978(1978-07-11)
- 11 мая 1992(1992-05-11)
- 12 января 1985(1985-01-12)
- 13 июля 1990(1990-07-13)
- 14 марта 1996(1996-03-14)
- 14 февраля 1954(1954-02-14)
- 14 февраля 1989(1989-02-14)
- 15 июня 1868(1868-06-15)
- 16 марта 2006(2006-03-16)
- 17 марта 1973(1973-03-17)
- 17 марта 2017 г.
- 17 января 2018(2018-01-17) (7 лет)
- 1738
- 1778(1778)
- 18 ноября 1987(1987-11-18)
- 1800(1800)
- 1825[1]
- 1830(1830)
- 1855(1855)
- 1859(1859)
- 19 января 1977(1977-01-19)
- 1911-1913
- 1914
- 1917 (примерно)
- 1930-е
- 1936 год
- 1941(1941)
- 1942 или 1941
- 1952(1952)
- 1954(1954)
- 1955(1955)
- 1957(1957)
- 1961
- 1963(1963)
- 1964–1967[1]
- 1970(1970)
- 1971 (?)
- 1972
- 1976[1]
- 1983 или 1984
- 1988(1988)
- 1991 (?)
- 1994(1994)
- 1996
- 2 ноября 2005(2005-

In [ ]:
import shutil
shutil.make_archive('архив', 'zip', 'famous_animals')

'/content/архив.zip'

### 1. Построить базу знаний в любой предметной области, сформировав массив документов (например, публикаций или файлов на каком-либо портале или набор файлов на компьютере или сервере и т.п.) и построить граф знаний об этих документах

In [ ]:
!pip install pyvis

In [ ]:
import os
import json
import networkx as nx
from pyvis.network import Network

In [ ]:
folder_path = 'famous_animals'

In [ ]:
G = nx.Graph()

In [ ]:
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            animal = data.get('Название', 'Без имени')
            animal_type = data.get('Вид животного', 'Неизвестно')
            birth_date = data.get('Дата рождения', 'Неизвестно')
            country = data.get('Страна', 'Неизвестно')

            # Узел животного
            G.add_node(animal, label=animal, title=f"Животное: {animal}\nВид: {animal_type}\nДата рождения: {birth_date}\nСтрана: {country}")

            # Узел страны
            if not G.has_node(country):
                G.add_node(country, label=country, title=f"Страна: {country}")

            G.add_edge(animal, country, relation='родом из')

            # Узел вида животного
            if not G.has_node(animal_type):
                G.add_node(animal_type, label=animal_type, title=f"Вид животного: {animal_type}")

            G.add_edge(animal, animal_type, relation='является')

        except Exception as e:
            print(f"Ошибка при обработке {filename}: {e}")

In [ ]:
net = Network(height="750px", width="100%", bgcolor="#ffffff", font_color="black", notebook=True)

net.from_nx(G)

# Включить физику для удобства перемещения
net.toggle_physics(True)

net.save_graph("famous_animals_graph.html")

### 2. Реализовать сервис краткого описания каждого документа в базе знаний (абстракт, автореферат - summary)

In [ ]:
def get_summary(animal_name, folder_path='famous_animals'):
    """
    Получает краткое описание животного по его имени файла.
    """
    filename = f"{animal_name}.json"
    path = os.path.join(folder_path, filename)

    if not os.path.exists(path):
        return "Файл не найден."

    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    name = data.get('Название', 'Неизвестно')
    animal_type = data.get('Вид животного', 'Неизвестно')
    country = data.get('Страна', 'Неизвестно')
    birth_date = data.get('Дата рождения', 'Неизвестно')
    description = data.get('Описание', 'Описание недоступно.')

    summary = (f"{name} — это {animal_type}, родом из {country}. "
               f"Родился в {birth_date}. Краткое описание: {description}")

    return summary

In [ ]:
print(get_summary("Пета"))

Пета — это Домашняя кошка, родом из Великобритания. Родился в октябрь 1963. Краткое описание: Пета (англ. Peta) — кошка, служившая главным мышеловом в кабинете правительства Соединенного Королевства в период с 1964 года и между 1969 и 1976 годами и была первой кошкой-самкой в этой роли. Она стала заменой Питеру III, который умер в возрасте 16 лет в 1964 году[1]. После его смерти, лейтенант-губернатор острова Мэн, сэр Рональд Гарви, предложил, чтобы Питера III заменила мэнская кошка и отправил Пету в кабинет. Она была ленивой и шумной, и к 1969 году некоторые государственные служащие пытались убрать её из кабинета министров, но этого не произошло из-за риска ухудшения репутации. Больше о ней не слышали, пока ответ представителю общественности в 1976 не показал, что её перевели в дом государственного служащего. Её преемником был Уилберфорс, который стал следующим Главным мышеловом в 1970-х годах[2].


### 3. Реализовать сервис поиска по запросу внутри базы знаний (вопрос-ответный модуль)

In [ ]:
def search_animals(query: str, folder_path='famous_animals'):
    """
    Ищет животных по запросу внутри базы знаний.
    Запрос ищет по названию, виду, стране и описанию.
    """
    results = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            path = os.path.join(folder_path, filename)

            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)

            name = data.get('Название', '').lower()
            animal_type = data.get('Вид животного', '').lower()
            country = data.get('Страна', '').lower()
            birth_date = data.get('Дата рождения', '').lower()
            description = data.get('Описание', '').lower()

            # если запрос содержится в любом из полей
            if (query.lower() in name or
                query.lower() in animal_type or
                query.lower() in country or
                query.lower() in birth_date or
                query.lower() in description):
                results.append(data)

    return results

In [ ]:
query = "Россия"
found = search_animals(query)

print(f"Найдено животных: {len(found)}")
for animal in found:
    print(f"- {animal['Название']} ({animal['Вид животного']}, {animal['Страна']})")

Найдено животных: 27
- Кузя (Тигр, подвид Уссурийский (амурский) тигр, Россия)
- Железный Ковбой (лошадь, Россия)
- Луга (Жираф, Россия)
- Арго (Волк, Россия)
- Большой Шлёпа (каракал, Россия)
- Анилин (Домашняя лошадь, Россия)
- Дик (Собака, Россия)
- Батыр (индийский слон, Россия)
- Земира (собака, Россия)
- Борец (Бурый медведь, Россия)
- Дина (Собака, Россия)
- Амадей (Тигр, Россия)
- Месси (Пума, Россия)
- Заратустра (Домашняя кошка, Россия)
- Крымский Том (Кошка, Россия)
- Абсент (дикая лошадь (подвид домашняя лошадь), Россия)
- Мальчик (собака, Россия)
- Марьям (Бурый медведь, Россия)
- Ань-Ань (Большая панда, Россия)
- Конни Полгрейв (Собака (Canis familiaris), Россия)
- Амур и Тимур (амурский тигр и козёл домашний, Россия)
- Лютый (Panthera tigris altaica, Россия)
- Маша (бурый медведь (европейский подвид), Россия)
- Золушка (Тигр, подвид Уссурийский (амурский) тигр, Россия)
- Микки (обыкновенный шимпанзе, Россия)
- Кинули (Лев, Россия)
- Лайка (Собака (Canis familiaris), Росс